Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [107]:
from random import choices, randint, random, choice
from copy import deepcopy
from tqdm.notebook import trange
import numpy as np



import lab9_lib

In [67]:
NUM_GENOMES = 1000

POPULATION_SIZE = 30
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = 0.15

In [89]:
class Individual:
    genomes: list
    fitness: float

    def __init__(self, genomes=None):
        if genomes == None:
            self.genomes = choices([0, 1], k=1000)
        else:
            self.genomes = genomes

    def mutate(self):
        mutated_genomes = deepcopy(self.genomes)
        index = choice(range(len(self.genomes)))
        if self.genomes[index] == 1:
            mutated_genomes[index] = 0
        else:
            mutated_genomes[index] = 1
        return Individual(genomes=mutated_genomes)


def select_parent(population) -> Individual:
    pool = choices(population, k=TOURNAMENT_SIZE)
    champion = max(pool, key=lambda i: i.fitness)
    return champion


def uniform_xover(ind1: Individual, ind2: Individual) -> Individual:
    offspring_genotype = [ind1.genomes[i] if random() < 0.5 else ind2.genomes[i] for i in range(NUM_GENOMES)]
    return Individual(genomes=offspring_genotype)


def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, NUM_GENOMES - 1)
    offspring = Individual(genomes=ind1.genomes[:cut_point] + ind2.genomes[cut_point:])
    return offspring

#### Problem 1

In [110]:
fitness = lab9_lib.make_problem(1)
population = [Individual() for _ in range(POPULATION_SIZE)]

for i in population:
    i.fitness = fitness(i.genomes)

for generation in range(3000):
    offspring = list()
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  # add more clever mutations
            p = select_parent(population)
            o = p.mutate()
        else:
            # xover # add more xovers
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = one_cut_xover(p1, p2)
        offspring.append(o)

    for i in offspring:
        i.fitness = fitness(i.genomes)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
    if population[0].fitness > 0.999:
        break
print(population[0].fitness)

1.0


In [111]:
print(fitness.calls)
best = population[0]

47370


### Problem 2
#### Promoting Diversity with **Extinction**

In [128]:
NUM_GENERATION = 10_000
PERCENTAGE_EXTINCTION = 0.85

POPULATION_SIZE = 50
OFFSPRING_SIZE = 25
TOURNAMENT_SIZE = 2
ONLY_MUTATION_PROBABILITY = 0.8
STD_THRESHOLD = 0.0005

In [129]:
fitness = lab9_lib.make_problem(2)
population = [Individual() for _ in range(POPULATION_SIZE)]


for i in population:
    i.fitness = fitness(i.genomes)

population.sort(key=lambda i: i.fitness, reverse=True)

best_individual = population[0]


pbar = trange(0, NUM_GENERATION)
for generation in pbar:
    pbar.set_description(f"Best-individual fitness: {best_individual.fitness:.2%}")
    offspring = list()

    # checking convergence
    population_fitness = [i.fitness for i in population]
    if np.std(population_fitness) < STD_THRESHOLD:
        num_indivual_to_extinction = int(POPULATION_SIZE * PERCENTAGE_EXTINCTION)
        population = choices(population, k=POPULATION_SIZE - num_indivual_to_extinction)
        offspring = [Individual() for _ in range(num_indivual_to_extinction)]

    else:
        for counter in range(OFFSPRING_SIZE):
            if random() < ONLY_MUTATION_PROBABILITY:  # self-adapt mutation probability
                # mutation  # add more clever mutations
                p = select_parent(population)
                o = p.mutate()
            else:
                # xover # add more xovers
                p1 = select_parent(population)
                p2 = select_parent(population)
                o = uniform_xover(p1, p2).mutate()

            offspring.append(o)

    for i in offspring:
        i.fitness = fitness(i.genomes)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]

    # save the new best individual
    if population[0].fitness > best_individual.fitness:
        best_individual = population[0]

print(population[0].fitness)

  0%|          | 0/10000 [00:00<?, ?it/s]

0.956


In [131]:
NUM_GENERATION = 20_000
PERCENTAGE_EXTINCTION = 0.9

POPULATION_SIZE = 80
OFFSPRING_SIZE = 30
TOURNAMENT_SIZE = 2
ONLY_MUTATION_PROBABILITY = 0.8
STD_THRESHOLD = 0.0005

In [132]:
fitness = lab9_lib.make_problem(2)
population = [Individual() for _ in range(POPULATION_SIZE)]


for i in population:
    i.fitness = fitness(i.genomes)

population.sort(key=lambda i: i.fitness, reverse=True)

best_individual = population[0]


pbar = trange(0, NUM_GENERATION)
for generation in pbar:
    pbar.set_description(f"Best-individual fitness: {best_individual.fitness:.2%}")
    offspring = list()

    # checking convergence
    population_fitness = [i.fitness for i in population]
    if np.std(population_fitness) < STD_THRESHOLD:
        num_indivual_to_extinction = int(POPULATION_SIZE * PERCENTAGE_EXTINCTION)
        population = choices(population, k=POPULATION_SIZE - num_indivual_to_extinction)
        offspring = [Individual() for _ in range(num_indivual_to_extinction)]

    else:
        for counter in range(OFFSPRING_SIZE):
            if random() < ONLY_MUTATION_PROBABILITY:  # self-adapt mutation probability
                # mutation  # add more clever mutations
                p = select_parent(population)
                o = p.mutate()
            else:
                # xover # add more xovers
                p1 = select_parent(population)
                p2 = select_parent(population)
                o = uniform_xover(p1, p2).mutate()

            offspring.append(o)

    for i in offspring:
        i.fitness = fitness(i.genomes)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]

    # save the new best individual
    if population[0].fitness > best_individual.fitness:
        best_individual = population[0]

  0%|          | 0/20000 [00:00<?, ?it/s]

0.998
